물론입니다. '날씨 요정'에서 배운 기초 위에 새로운 개념을 쌓아 올릴 '두 번째 에이전트, 환율 계산기 로봇' 챕터를 구체적으로 작성해 드리겠습니다.

-----

### **Chapter 4: 두 번째 에이전트, '환율 계산기 로봇'**

'날씨 요정' 프로젝트를 통해 우리는 AI 에이전트의 핵심 작동 원리를 성공적으로 경험했습니다. 정말 대단한 첫걸음이었죠\! 이번 챕터에서는 그 성공 경험을 바탕으로, 우리 에이전트의 능력을 한 단계 더 업그레이드해 보겠습니다.

이번에 만들 에이전트는 '환율 계산기 로봇'입니다. 이 과정을 통해 여러분은 우리 에이전트가 얼마나 더 똑똑하고 유연해질 수 있는지 확인하게 될 겁니다.

가장 좋은 소식은, **기본적인 구조는 '날씨 요정'과 완전히 똑같다는 점**입니다. 우리는 단지 AI에게 새로운 '도구'를 쥐여주기만 하면 됩니다. 이전 챕터에서 쌓은 지식이 그대로 사용되니, 훨씬 편안하게 따라오실 수 있을 거예요.

#### **🎯 목표: 여러 정보를 알아서 처리하는 똑똑한 환율 계산기**

이번 챕터의 목표는 사용자의 질문 하나에서 여러 핵심 정보를 스스로 뽑아내어 계산을 수행하는 에이전트를 만드는 것입니다.

> **우리의 목표:** "150달러는 한국 돈으로 얼마야?" 와 같은 질문을 던졌을 때, AI가 **'150'**, **'달러'**, \*\*'한국 돈'\*\*이라는 3가지 핵심 정보를 스스로 추출하고, 이를 바탕으로 정확한 환율 계산 결과를 알려주게 하는 것입니다.

'날씨 요정'이 단 하나의 정보('도시 이름')만 필요로 했다면, 이번에는 3개의 정보를 동시에 처리해야 합니다. 과연 AI가 이 복잡한 임무를 잘 해낼 수 있을까요?

#### **새로운 도구와 '업그레이드된 신청서'**

환율을 계산하려면, '날씨 검색 로봇'과는 다른 새로운 '미니 로봇(함수)'이 필요합니다. 이 로봇은 3가지 정보(금액, 바꾸기 전 통화, 바꿀 통화)를 받아야만 작동할 수 있습니다.

##### **새로운 '미니 로봇': `get_exchange_rate` 함수**

```python
# 3가지 정보를 받아 환율을 계산하는 '환율 계산기 로봇'을 만듭니다.
def get_exchange_rate(amount, from_currency, to_currency):
    """지정된 금액을 한 통화에서 다른 통화로 변환하는 가짜 함수입니다."""
    print(f"🛠️ [도구 실행] 'get_exchange_rate' 로봇이 {amount} {from_currency}를 {to_currency}로 변환합니다.")
    
    # 간단한 실습을 위해 환율을 미리 정해둡니다.
    exchange_rates = {
        "USD_to_KRW": 1300,
        "EUR_to_KRW": 1400,
    }
    
    conversion_key = f"{from_currency}_to_{to_currency}"
    rate = exchange_rates.get(conversion_key)
    
    if rate:
        converted_amount = amount * rate
        return json.dumps({"amount": converted_amount, "currency": to_currency})
    else:
        return json.dumps({"error": "지원하지 않는 통화 변환입니다."})
```

##### **'업그레이드된 도구 등록 신청서'**

3개의 정보가 필요하므로, AI에게 제출할 '도구 등록 신청서'도 더 상세해져야 합니다. `parameters` 항목이 어떻게 바뀌었는지 유심히 살펴보세요.

```python
# '환율 계산기 로봇'의 '도구 등록 신청서'
tools = [{
    "type": "function",
    "function": {
        "name": "get_exchange_rate",
        "description": "한 통화의 금액을 다른 통화로 환전해줍니다.",
        # '날씨 요정'과 달리, 3개의 정보가 필요하다고 알려줍니다.
        "parameters": {
            "type": "object",
            "properties": {
                # 1. 금액 정보
                "amount": {
                    "type": "number",
                    "description": "환전할 금액"
                },
                # 2. 시작 통화 정보
                "from_currency": {
                    "type": "string",
                    "description": "환전의 기준이 되는 통화 (예: USD, EUR)"
                },
                # 3. 목표 통화 정보
                "to_currency": {
                    "type": "string",
                    "description": "환전 후의 목표 통화 (예: KRW)"
                }
            },
            # 3가지 정보 모두 필수!
            "required": ["amount", "from_currency", "to_currency"],
        },
    }
}]
```

#### **'환율 계산기 로봇' 전체 코드 및 실행**

자, 이제 전체 코드를 통해 '환율 계산기 로봇'이 어떻게 작동하는지 확인해 보겠습니다. **주석은 '날씨 요정'과 달라진 부분, 즉 새로운 도구와 관련된 부분에 집중**되어 있습니다. 이미 익숙한 부분은 주석을 최소화했으니, 변경된 부분을 중심으로 코드를 살펴보세요.

```python
import openai
import json

# --- 준비 과정 (날씨 요정과 동일) ---
client = openai.OpenAI(api_key="여러분의_API_키")

# --- 새로운 도구 제작: '환율 계산기 로봇' ---
# 3가지 정보(금액, 시작 통화, 목표 통화)를 받아 계산하는 점이 다릅니다.
def get_exchange_rate(amount, from_currency, to_currency):
    """지정된 금액을 한 통화에서 다른 통화로 변환하는 가짜 함수입니다."""
    print(f"🛠️ [도구 실행] 'get_exchange_rate' 로봇이 {amount} {from_currency}를 {to_currency}로 변환합니다.")
    exchange_rates = {"USD_to_KRW": 1300, "EUR_to_KRW": 1400}
    conversion_key = f"{from_currency}_to_{to_currency}"
    rate = exchange_rates.get(conversion_key)
    if rate:
        converted_amount = amount * rate
        return json.dumps({"amount": converted_amount, "currency": to_currency})
    else:
        return json.dumps({"error": "지원하지 않는 통화 변환입니다."})

# --- 에이전트 실행 (핵심 구조는 날씨 요정과 동일) ---
def run_exchange_agent(user_question):
    print(f"😎 사용자 질문: {user_question}\n")
    print("🧠 [1차 호출] AI, 이 질문을 보고 필요한 정보를 뽑아서 도구를 쓸지 결정해줘!")
    
    # 여기서는 '환율 계산기'의 '업그레이드된 신청서'를 제출합니다.
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": user_question}],
        tools=[{
            "type": "function",
            "function": {
                "name": "get_exchange_rate",
                "description": "한 통화의 금액을 다른 통화로 환전해줍니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "amount": {"type": "number", "description": "환전할 금액"},
                        "from_currency": {"type": "string", "description": "환전의 기준이 되는 통화 (예: USD, EUR)"},
                        "to_currency": {"type": "string", "description": "환전 후의 목표 통화 (예: KRW)"}
                    },
                    "required": ["amount", "from_currency", "to_currency"],
                },
            }
        }],
        tool_choice="auto",
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        # AI가 하나의 문장에서 3개의 정보를 정확히 추출했는지 확인해보세요!
        print("✅ AI 응답: '환율 계산기 로봇'을 사용해야겠어요!")
        tool_call = tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        print(f"   - AI가 추출한 정보: {function_args}\n")

        # 이하 도구 실행 및 2차 호출 과정은 날씨 요정과 완전히 동일합니다.
        messages = [{"role": "user", "content": user_question}, response_message]
        function_response = get_exchange_rate(
            amount=function_args.get("amount"),
            from_currency=function_args.get("from_currency"),
            to_currency=function_args.get("to_currency"),
        )
        print(f"📄 [도구 결과] 계산 결과: {function_response}\n")
        
        print("🧠 [2차 호출] AI, 계산 결과를 바탕으로 최종 답변을 만들어줘!")
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": "get_exchange_rate",
            "content": function_response,
        })
        second_response = client.chat.completions.create(model="gpt-4o", messages=messages)
        return second_response.choices[0].message.content
    else:
        return response_message.content

# --- 최종 실행 및 성공 확인 ---
final_result = run_exchange_agent("150달러는 한국 돈으로 얼마야?")
print("\n🎉 [최종 답변] 환율 계산기 로봇의 답변입니다!")
print(final_result)
```

#### **성공 확인 및 핵심 포인트**

위 코드를 실행하면, 터미널에 다음과 같은 놀라운 결과를 확인하실 수 있습니다.

> ✅ AI 응답: '환율 계산기 로봇'을 사용해야겠어요\!
>
>   - AI가 추출한 정보: `{'amount': 150, 'from_currency': 'USD', 'to_currency': 'KRW'}`

보셨나요? AI는 "150달러는 한국 돈으로 얼마야?"라는 한 문장에서, 우리가 가르쳐준 대로 `amount`, `from_currency`, `to_currency`라는 3개의 정보를 마치 탐정처럼 정확하게 찾아냈습니다.

최종적으로는 다음과 같은 깔끔한 답변을 얻게 됩니다.

> 🎉 **[최종 답변] 환율 계산기 로봇의 답변입니다\!**
>
> 150달러는 195000원입니다.

이번 챕터를 통해 우리는 에이전트의 패턴이 얼마나 강력하고 유연한지 확인했습니다. 도구의 '신청서'만 잘 작성해주면, AI는 아무리 복잡한 정보를 요구하는 도구라도 척척 해낼 수 있습니다. 이제 우리는 날씨 확인은 물론 환율 계산까지 가능한, 한층 더 유능해진 나만의 비서를 갖게 되었습니다\!